In [8]:

from nltk.tokenize import word_tokenize
import csv
import threading, json
import string
from nltk.corpus import stopwords
from collections import defaultdict
from multiprocessing import Pool, cpu_count

# Function to tokenize the title using NLTK
def tokenize(title):
    return word_tokenize(title.lower())

# Function to write an inverted index to a file in the specified JSON format
def write_inverted_index_to_json(inverted_index, filename):
    with open(filename, 'w') as file:
        for term in sorted(inverted_index.keys()):
            documents = inverted_index[term]
            data = {
                "term": term,
                "documents": documents,
                "document_count": len(documents)
            }
            json_line = json.dumps(data)
            file.write(json_line + '\n')

# Function to process a chunk of rows and create an inverted index
def process_chunk(chunk, chunk_id):
    print(f"Processing chunk {chunk_id}...")
    inverted_index = defaultdict(set)
    for row in chunk:
        tokens = tokenize(row[2].lower())  # Assuming 'primaryTitle' is the third column
        for token in tokens:
            if token in string.punctuation or token in stopwords.words('english'):
                continue
            inverted_index[token].add(row[0])  # Assuming 'tconst' is the first column
    
    # Convert sets to lists for better readability
    for token in inverted_index:
        inverted_index[token] = list(inverted_index[token])

    # Write the inverted index of this chunk to a file
    write_inverted_index_to_json(inverted_index, f'intermediate_v1/inverted_index_chunk_{chunk_id}.json')

# Read the TSV file and divide it into chunks
chunks = []
chunk_size = 1  # Define your chunk size
pool = Pool(cpu_count())  # Create a multiprocessing Pool
# Assuming you've defined and incremented chunk_id somewhere
chunk_id = 0
results = []

with open('Original_datasets/title.basics.tsv', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    next(reader, None)  # Skip the header line

    chunk = []
    for i, row in enumerate(reader):
        if i == 2: break
        chunk.append(row)
        if len(chunk) == chunk_size:
            result = pool.apply_async(process_chunk, args=(chunk, chunk_id))
            results.append(result)
            chunk_id += 1
            chunk = []

    if chunk:  # Process the last chunk if it has any rows
        result = pool.apply_async(process_chunk, args=(chunk, chunk_id))
        results.append(result)

# Wait for all tasks to complete
for result in results:
    result.get()  # This will re-raise any exceptions that occurred

pool.close()
pool.join()

print("Processing complete.")


Process SpawnPoolWorker-58:
Process SpawnPoolWorker-57:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/uditsankhadasariya/opt/anaconda3/envs/myenv/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/uditsankhadasariya/opt/anaconda3/envs/myenv/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/uditsankhadasariya/opt/anaconda3/envs/myenv/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/Users/uditsankhadasariya/opt/anaconda3/envs/myenv/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_chunk' on <module '__main__' (built-in)>
  File "/Users/uditsankhadasariya/opt/anaconda3/envs/myenv/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    